In [12]:
import cv2
import numpy as np
import HandTrackingModule as htm
import time
import autopy

In [13]:
wCam,hCam = 640,480
frameR = 100 # frame reduction
wScr = 1920 
hScr = 1080
smoothening = 6
print(wScr)
print(hScr)

1920
1080


In [14]:
pTime = 0
pLocX, pLocY = 0,0
cLocX, cLocY = 0,0

detector = htm.handDetector(maxHands=1)

In [15]:
cap = cv2.VideoCapture(0)
cap.set(3, wCam)
cap.set(4, hCam)

while True:
    #1. Find hand landmark
    # 1. Find hand landmark
    success, img = cap.read()
    
    # Check if the image was captured successfully
    if not success or img is None:
        print("Failed to capture image")
        break

    img = detector.findHands(img)
    lmList, bbox = detector.findPosition(img)
 
    #2. Get the tip of index and middle finger
    if len(lmList) != 0:
        x1, y1 = lmList[8][1:]
        x2, y2 = lmList[12][1:]
        
        #print(x1,y1,x2,y2)
    
    #3. Check which finger are up
    fingers = detector.fingersUp()
    #print(fingers)
    #4. Only Index finger: Moving mode condition define
    cv2.rectangle(img,(frameR,frameR),(wCam - frameR, hCam -frameR), (255,0,255), 2)
    if len(fingers) >= 3 and fingers[1] == 1 and fingers[2] == 0:
        #print("ok")
        #5. Conver coordinate (screen resolution depends)
        x3 = np.interp(x1, (frameR, wCam - frameR),(0,wScr))
        y3 = np.interp(y1, (frameR, hCam - frameR),(0,hScr))
        #6. Smooth the value
        cLocX = pLocX + (x3-pLocX) /smoothening
        cLocY = pLocY + (y3-pLocY) /smoothening
        #7. move mouse
        autopy.mouse.move(wScr - cLocX, cLocY)
        cv2.circle(img, (x1,y1), 15, (255,0,255), cv2.FILLED) #moving mode enable recognition
        pLocX,pLocY = cLocX, cLocY


    
    #8 Both finger up = clicking mode
    if len(fingers) >= 3 and fingers[1] == 1 and fingers[2] == 1:
        length, img, lineInfo = detector.findDistance(8, 12, img)
        #print(length)
        #9. Check Distance
        if length < 16:
            cv2.circle(img, (lineInfo[4],lineInfo[5]), 15, (0,255,0), cv2.FILLED)
            #10 Click mouse if distance short
            autopy.mouse.click()
        #5. Conver coordinate (screen resolution depends)
        x3 = np.interp(x1, (frameR, wCam - frameR),(0,wScr))
        y3 = np.interp(y1, (frameR, hCam - frameR),(0,hScr))
        #6. Smooth the value
        cLocX = pLocX + (x3-pLocX) /smoothening
        cLocY = pLocY + (y3-pLocY) /smoothening
        #7. move mouse
        autopy.mouse.move(wScr - cLocX, cLocY)
        cv2.circle(img, (x1,y1), 15, (255,0,255), cv2.FILLED) #moving mode enable recognition
        pLocX,pLocY = cLocX, cLocY
    
    
    #11 Frame rate
    cTime = time.time()
    fps = 1/(cTime-pTime)
    pTime = cTime
    cv2.putText(img,str(int(fps)),(20,50), cv2.FONT_HERSHEY_PLAIN,3,(255,0,0),3)
    
    #12 display
    cv2.imshow("image", img)

    # Exit the loop on ESC key press
    k = cv2.waitKey(1) & 0xFF
    if k == 27:  # Escape key
        break

# Release the camera and close the windows
cap.release()
cv2.destroyAllWindows()
